In [1]:

!pip install playwright nest_asyncio
!playwright install chromium
!apt-get install libatk1.0-0 libatk-bridge2.0-0 libatspi2.0-0 libxcomposite1


import asyncio, json, csv
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from playwright.async_api import async_playwright



#  3. BASE URL
BASE_URL = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops"

# 4. MAIN SCRAPING FUNCTION
async def scrape_ajax_site():

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)   # Launch browser
        ctx = await browser.new_context()                  # New browser session
        page = await ctx.new_page()                        # New tab

        rows = []                                         # Stores ALL products
        page_no = 1                                       #  Start from page 1

        # 5. PAGE NUMBER LOOP (1 → 20)
        while True:
            url = f"{BASE_URL}?page={page_no}"             #  Build page URL
            print(f"Scraping Page {page_no} → {url}")
            await page.goto(url, timeout=60000)            #  Open that page
            try:
                await page.wait_for_selector(".thumbnail", timeout=10000)
                #  Wait for product cards
            except:
                print(" No more pages left. Stopping.")
                break                                     #  Stop when no products found
            cards = await page.query_selector_all(".thumbnail")

            if not cards:                                 # Safety stop
                print(" Last page reached.")
                break
            #  Extract products from the CURRENT page
            for card in cards:

                title_el = await card.query_selector(".title")
                title = (await title_el.text_content()).strip() if title_el else None

                url = await title_el.get_attribute("href") if title_el else None

                price_el = await card.query_selector(".price")
                price = (await price_el.text_content()).strip() if price_el else None

                stars = await card.query_selector_all(".ratings .glyphicon-star")
                rating = len(stars) if stars else 0

                img_el = await card.query_selector("img")
                img_src = await img_el.get_attribute("src") if img_el else None

                rows.append({
                    "title": title,
                    "price": price,
                    "rating_stars": rating,
                    "product_url": url,
                    "image_url": img_src,
                    "page_no": page_no
                })

            page_no += 1                                   # Go to next page number

        await browser.close()
        return rows

#  6. RUN SCRAPER

data = asyncio.get_event_loop().run_until_complete(scrape_ajax_site())
print(f" Collected {len(data)} total products")


# 7. SAVE OUTPUT FILES

Path("ioutput").mkdir(exist_ok=True)                       #  Create output folder

csv_path = Path("ioutput/products_all_ajax.csv")
json_path = Path("ioutput/products_all_ajax.json")

# Save CSV
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

# Save JSON
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Saved CSV → {csv_path}")
print(f"Saved JSON → {json_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 7.3 MB/s eta 0:00:00
(node:373) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 44.5s164.7 MiB [] 0% 86.5s164.7 MiB [] 0% 26.0s164.7 MiB [] 0% 20.0s164.7 MiB [] 0% 20.4s164.7 MiB [] 0% 15.1s164.7 MiB [] 1% 10.5s164.7 MiB [] 1% 8.2s164.7 MiB [] 1% 7.4s164.7 MiB [] 2% 6.5s164.7 MiB [] 2% 5.9s164.7 MiB [] 3% 5.5s164.7 MiB [] 3% 5.2s164.7 MiB [] 4% 5.4s164.7 MiB [] 4% 5.8s164.7 MiB [] 4% 5.5s164.7 MiB [] 5% 5.3s164.7 MiB [] 6% 5.0s164.7 MiB [] 6% 4.7s164.7 MiB [] 7% 4.6s164.7 MiB [] 7% 4.5s164.7 MiB [] 8% 4.3s164.7 MiB [] 9% 4.2s164.7 MiB [] 9% 4.1s164.7 MiB [] 10% 4.0s164.7 MiB [] 10% 3.9s164.7 MiB [] 11% 3.7s164.7 MiB [] 12% 3.6s164.7 MiB [] 13% 3.6s164.7 MiB [

📌 Observations – Page-Based Pagination Web Scraping

🔹 1. Pagination Handling

The scraper uses URL-based pagination by incrementing the page query parameter (?page=1, ?page=2, …).

This confirms that the target website supports static server-side pagination, not infinite scroll or AJAX-only loading.

Scraping stops automatically when:

No product cards (.thumbnail) are found, or

The selector wait times out.

Observation:
This method is reliable and deterministic because each page has a fixed URL and complete HTML content.


🔹 2. Scraping Strategy

The script opens one browser session and one page (tab).

The same page object is reused for all page numbers.

For each page:

All product cards are selected.

Product details are extracted from the listing page itself, without visiting individual product pages.

Observation:
This approach is efficient and avoids unnecessary navigation, reducing scraping time and browser overhead.

🔹 3. Extracted Data Fields

For every product, the following information is collected:

Title – Product name

Price – Displayed product price

Rating (Stars) – Derived by counting star icons

Product URL – Link to product detail page

Image URL – Product image source

Page Number – Page from which the product was scraped

Observation:
The dataset is well-structured and suitable for further analysis such as price comparison, rating analysis, or visualization.

🔹 4. Rating Extraction Logic

Product ratings are calculated by counting the number of .glyphicon-star elements.

This avoids reliance on text values and ensures consistent numeric ratings.

Observation:
Using DOM element counts for ratings is robust and less error-prone than parsing text.

🔹 5. Loop Termination Condition

The scraper safely exits when:

The selector .thumbnail is not found, or

An empty product list is returned.

Observation:
This prevents infinite loops and ensures graceful termination when the last page is reached.

🔹 6. Performance Considerations

Page-based navigation is faster than AJAX “Load More” approaches.

No JavaScript event handling or network-idle polling is required.

Observation:
This method scales well for large datasets and is ideal for production-grade scraping.

🔹 7. Output Generation

Data is saved in both CSV and JSON formats.

Output directory is created safely using pathlib.

Observation:
Providing both formats increases usability for data analysis, machine learning, and reporting.

🔹 8. Reliability & Maintainability

The scraper logic is simple, readable, and modular.

Fewer failure points compared to dynamic/AJAX scraping.

Observation:
This approach is highly maintainable and suitable for academic projects, internships, and interviews.

✅ Final Conclusion

This scraper successfully demonstrates page-based pagination scraping, efficiently collecting structured product data from a static e-commerce website. The approach is reliable, fast, and well-suited for large-scale data collection, making it preferable over dynamic “Load More” scraping when URL-based pagination is available.